# Introduction to Kokkos

This lab will introduce the Kokkos framework for portable performance using both OpenMP and CUDA
as a backend.

The examples follow the first few of a longer sequence of tutorial examples appearing in the SANDIA Kokkos tutorial found at 
https://github.com/kokkos/kokkos-tutorials

The full Kokkos programming guide can be found at https://github.com/kokkos/kokkos/wiki

Instructions are provided below (and within the templates) to allow you to complete each of the exercises.

# (1) Initialise, Use, Finalise

The first exercise asks you to initialise and finalise Kokkos, and use the patterns `parallel_for` and `parallel_reduce` in the computation of an inner product.

The corresponding template source code is in subdirectory `1`; ie., `1/exercise_1_begin.cpp`. Use the notebook navigation to open this file in the editor. The places where you need to change things are marked with "EXERCISE".

You will need to recall the patterns:
```cpp
  Kokkos::parallel_for(N, KOKKOS_LAMBDA (int i) {
    //... loop body with index i of extent N
  });
```
where the macro `KOKKOS_LAMBDA` is representing the capture. The `parallel_reduce` pattern is of the form
```cpp
  Kokkos::parallel_reduce(M, KOKKOS_LAMBDA (int j, double & sum) {
    // sum += ...loop body with index j of extent M
  }, result);
```
where `result` is a variable defined in the outer scope to hold the result. The `sum` variable is managed by Kokkos.


You can compile the code using the OpenMP backend with the below cell.

In [ ]:
! cd 1; make OpenMP; cd ..

When you run you can set the problem size with command line flags. These are passed as a power of 2 (e.g. `10 => 2**10 = 1024`).
* `-N` - the number of rows (default = 12)
* `-M` - the number of columns (default = 10)
* `-S` - the total size (default = 22, must equal sum of M and N)

Can also specify the number of repeats:
* `-nrepeat` (default = 100)

When using OpenMP, specify the number of threads to use by setting the `OMP_NUM_THREADS` environment variable.

In [ ]:
! cd 1; env OMP_NUM_THREADS=1 OMP_PROC_BIND=spread OMP_PLACES=threads ./01_Exercise.OpenMP -N 12; cd ..

# (2) Use Views

The goal of this exercise will be to replace the raw memory allocations with Kokkos Views, and corrresponding memory accesses
with the relevant Kokkos view access.

The source code is found in `2/exercise_2_begin.cpp`.
Again, code requiring attention is marked with 'EXERCISE'.

Recall that a View may be declared via, e.g.,
```cpp
Kokkos::View < double * > x("my vector", nElements);
```
and access to individual elements is via brackets, e.g.,
```cpp
x(index) = ...
```

Note we force Kokkos use to UVM for the CUDA build.

In [ ]:
# Execute this cell to compile the code
! cd 2; make; cd ..

In [ ]:
# Execute this cell to run the code on the CPU with OpenMP
# You can vary the number of threads used with OMP_NUM_THREADS (note this machine has 4 cores)
! cd 2; env OMP_NUM_THREADS=4 OMP_PROC_BIND=spread OMP_PLACES=threads ./02_Exercise.OpenMP -S 26; cd ..

In [ ]:
# Execute this cell to run the code on the GPU with Cuda + UVM
! cd 2; env CUDA_LAUNCH_BLOCKING=1 CUDA_VISIBLE_DEVICES=0 ./02_Exercise.CudaUVM ; cd ..

In [ ]:
import sys
sys.version
sys.path

In [ ]:
from matplotlib import pylab as plt
import numpy as np
%matplotlib inline

In [ ]:
# Add results and run this cell to view a plot

# Example data - replace with your measurements!
# GPU/CPU, S, N, bandwidth/(GB/s)
data = [('gpu',26, 16, 14.3),
        ('cpu',26, 16, 56.9)]

def plot():
    d = np.array(data,
                 dtype=[('type', object), ('log2size', int), ('log2rows', int), ('bandwidth', float)])
    cpu = d[np.where(d['type']=='cpu')]; gpu = d[np.where(d['type']=='gpu')]
    
    ax = plt.gca()
    ax.semilogx()
    ax.set_ylim(bottom=0.0, top=d['bandwidth'].max())

    ax.plot(2**cpu['log2rows'], cpu['bandwidth'], 'bx',
            2**gpu['log2rows'], gpu['bandwidth'], 'rx')
plot()

# (3) Use Mirror Views

Now, we will replace use of managed GPU memory with explicit
data management via Kokkos mirror views and and copies.

The exercise template is `3/exercise_3_begin.cpp`

Recall the copy is
```cpp
Kokkos::deep_copy(dest, src);
```
for the required direction of transfer.

In [ ]:
# Execute this cell to compile the code
! cd 3; make; cd ..

In [ ]:
# Execute this cell to run the code on the CPU with OpenMP
! cd 3; env OMP_NUM_THREADS=1 OMP_PROC_BIND=spread OMP_PLACES=threads ./03_Exercise.OpenMP ; cd ..

In [ ]:
# Execute this cell to run the code on the GPU
! cd 3; ./03_Exercise.Cuda ; cd ..

In [ ]:
# Add results and run this cell to view a plot

# Example data - replace with your measurements!
# GPU/CPU, S, N, bandwidth/(GB/s)
data = [('gpu',26, 16, 14.3),
        ('cpu',26, 16, 56.9)]

def plot():
    d = np.array(data,
                 dtype=[('type', object), ('size', int), ('rows', int), ('bandwidth', float)])
    d['size'] = 2**d['size']
    d['rows'] = 2**d['rows']

    cpu = d[np.where(d['type']=='cpu')]; gpu = d[np.where(d['type']=='gpu')]
    
    ax = plt.gca()
    ax.semilogx()
    ax.set_ylim(bottom=0.0, top=d['bandwidth'].max())

    ax.plot(cpu['rows'], cpu['bandwidth'], 'bx',
            gpu['rows'], gpu['bandwidth'], 'rx')
plot()

# (4) Control the Layout

The final exercise provides some wide-ranging options to investigate memory layouts. memory and execution spaces; using a
Kokkos RangePolicy to parallelise the inner loop.

The template is `4/exercise_4_begin.cpp` contains further instructions and hints.


In [ ]:
# Execute this cell to compile the code
! cd 4; make; cd ..

In [ ]:
# Execute this cell to run the code
# You may wish to vary OMP_NUM_THREADS when running on the CPU
! cd 4; env OMP_NUM_THREADS=4 OMP_PROC_BIND=spread OMP_PLACES=threads ./04_Exercise.Any ; cd ..

In [ ]:
# Add results and run this cell to view a plot

# Example data - replace with your measurements!
# GPU/CPU, Layout, S, N, bandwidth/(GB/s)
data = [('gpu', 'left', 26, 16, 14.3),
        ('cpu', 'right', 26, 16, 56.9)]

def plot():
    d = np.array(data,
                 dtype=[('type', object), ('layout', object), ('size', int), ('rows', int), ('bandwidth', float)])
    d['size'] = 2**d['size']
    d['rows'] = 2**d['rows']
    
    cpu_left = d[np.where(np.logical_and(d['type']=='cpu',d['layout']=='left'))]
    cpu_right = d[np.where(np.logical_and(d['type']=='cpu',d['layout']=='right'))]
    gpu_left = d[np.where(np.logical_and(d['type']=='gpu',d['layout']=='left'))]
    gpu_right = d[np.where(np.logical_and(d['type']=='gpu',d['layout']=='right'))]
    
    ax = plt.gca()
    ax.semilogx()
    ax.set_ylim(bottom=0.0, top=d['bandwidth'].max())

    ax.plot(cpu_left['rows'], cpu_left['bandwidth'], 'bx',
            gpu_left['rows'], gpu_left['bandwidth'], 'rx',
            cpu_right['rows'], cpu_right['bandwidth'], 'bo',
            gpu_right['rows'], gpu_right['bandwidth'], 'ro')
plot()